<a href="https://colab.research.google.com/github/binliu0630/Data_Manipulation/blob/master/Copy_of_2020_03_21_covid19_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [0]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [0]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dft_recovered.columns[-1]].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dft_recovered.columns[-6]].sum()

In [0]:
dfc_recovered.head(2)

Country/Region
Afghanistan     1
Albania        10
Name: 3/24/2020, dtype: int64

In [0]:
dfc_deaths.head(2)

Country/Region
Afghanistan    2
Albania        5
Name: 3/25/20, dtype: int64

In [0]:
dfc_recovered.head(2)

Country/Region
Afghanistan     1
Albania        10
Name: 3/24/2020, dtype: int64

In [0]:
# Add US STATE AND COUNTY
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-25-2020.csv'
df = pd.read_csv(url)


In [0]:
data = df[(df['Country_Region']=='US')&(df['Province_State'].isin(['Arkansas','New York', 'Louisiana']))]
df_state = data.groupby(['Province_State']).agg({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum'})
df_state

,Confirmed,Deaths,Recovered
Province_State,,,
Arkansas,280,2,0
Louisiana,1795,65,0
New York,30841,285,0


In [0]:
dfc_cases

Country/Region
Afghanistan            84
Albania               146
Algeria               302
Andorra               188
Angola                  3
                     ... 
Venezuela              91
Vietnam               141
West Bank and Gaza     -1
Zambia                 12
Zimbabwe                3
Name: 3/25/20, Length: 174, dtype: int64

In [0]:
df_state['Confirmed']

Province_State
Arkansas       280
Louisiana     1795
New York     30841
Name: Confirmed, dtype: int64

In [0]:
data.groupby(['Province_State']).agg({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum'})

,Confirmed,Deaths,Recovered
Province_State,,,
Arkansas,280,2,0
New York,30841,285,0


In [0]:
data.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
27,36001.0,Albany,New York,US,2020-03-25 23:33:19,42.600603,-73.977239,152,0,0,0,"Albany, New York, US"
42,36003.0,Allegany,New York,US,2020-03-25 23:33:19,42.257484,-78.027505,2,0,0,0,"Allegany, New York, US"
82,5001.0,Arkansas,Arkansas,US,2020-03-25 23:33:19,34.291452,-91.372773,0,0,0,0,"Arkansas, Arkansas, US"
92,5003.0,Ashley,Arkansas,US,2020-03-25 23:33:19,33.191535,-91.769847,0,0,0,0,"Ashley, Arkansas, US"
149,5005.0,Baxter,Arkansas,US,2020-03-25 23:33:19,36.287844,-92.337829,0,0,0,0,"Baxter, Arkansas, US"


In [0]:
dft_cases['Province/State'].value_counts()

Ontario             1
Bermuda             1
Liaoning            1
Martinique          1
Guadeloupe          1
                   ..
New South Wales     1
Gansu               1
Guangdong           1
British Columbia    1
Jilin               1
Name: Province/State, Length: 74, dtype: int64

In [0]:
print(list(dfc_cases.index))

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', '

In [0]:
dfc_cases[dfc_cases.index.isin(['China', 'US'])]

Country/Region
China    81661
US       65778
Name: 3/25/20, dtype: int64

In [0]:
#hide
dfc_recovered = dft_cases.copy()
dfc_recovered.loc[:, dft_cases.columns[4:]] = 0
dfc_recovered

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,Libya,26.335100,17.228331,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
241,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
242,NaN,Guinea-Bissau,11.803700,-15.180400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
243,NaN,Mali,17.570692,-3.996166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())

In [0]:
df_table

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81661.0,3285.0,73280.0,81250.0,3253.0,70535.0,411.0,32.0,2745.0,4.0,Asia
1,Italy,74386.0,7503.0,8326.0,47021.0,4032.0,4440.0,27365.0,3471.0,3886.0,10.1,Europe
2,US,65778.0,942.0,348.0,19100.0,244.0,121.0,46678.0,698.0,227.0,1.4,North America
3,Spain,49515.0,3647.0,3794.0,20410.0,1043.0,1107.0,29105.0,2604.0,2687.0,7.4,Europe
4,Germany,37323.0,206.0,3243.0,19848.0,67.0,113.0,17475.0,139.0,3130.0,0.6,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
169,Papua New Guinea,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
170,Saint Vincent and the Grenadines,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,North America
171,Somalia,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Africa
172,Timor-Leste,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN


In [0]:
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)

In [0]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81661.0,3285.0,73280.0,81250.0,3253.0,70535.0,411.0,32.0,2745.0,4.0,Asia
1,Italy,74386.0,7503.0,8326.0,47021.0,4032.0,4440.0,27365.0,3471.0,3886.0,10.1,Europe
2,US,65778.0,942.0,348.0,19100.0,244.0,121.0,46678.0,698.0,227.0,1.4,North America
3,Spain,49515.0,3647.0,3794.0,20410.0,1043.0,1107.0,29105.0,2604.0,2687.0,7.4,Europe
4,Germany,37323.0,206.0,3243.0,19848.0,67.0,113.0,17475.0,139.0,3130.0,0.6,Europe
5,Iran,27017.0,2077.0,8913.0,19644.0,1433.0,5710.0,7373.0,644.0,3203.0,7.7,Asia
6,France,25600.0,1333.0,3250.0,12758.0,451.0,12.0,12842.0,882.0,3238.0,5.2,Europe
7,Switzerland,10897.0,153.0,131.0,5294.0,54.0,15.0,5603.0,99.0,116.0,1.4,Europe
8,United Kingdom,9640.0,466.0,140.0,4014.0,178.0,67.0,5626.0,288.0,73.0,4.8,Europe
9,South Korea,9137.0,126.0,3507.0,8652.0,94.0,1540.0,485.0,32.0,1967.0,1.4,Asia


In [0]:
#hide

#delete problematic countries from table
df_table = df_table[df_table['Country/Region'] != 'Cape Verde']
df_table = df_table[df_table['Country/Region'] != 'Cruise Ship']
df_table = df_table[df_table['Country/Region'] != 'Kosovo']

In [0]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'Cases': 467593.0,
 'Cases (+)': 195608.0,
 'China Cases': 81661.0,
 'China Cases (+)': 411.0,
 'China Deaths': 3285.0,
 'China Deaths (+)': 32.0,
 'China Recovered': 73280.0,
 'China Recovered (+)': 2745.0,
 'Deaths': 21180.0,
 'Deaths (+)': 9886.0,
 'EU Cases': 248822.0,
 'EU Cases (+)': 119470.0,
 'EU Deaths': 14177.0,
 'EU Deaths (+)': 8116.0,
 'EU Recovered': 19756.0,
 'EU Recovered (+)': 13846.0,
 'Recovered': 107659.0,
 'Recovered (+)': 23018.0,
 'US Cases': 65778.0,
 'US Cases (+)': 46678.0,
 'US Deaths': 942.0,
 'US Deaths (+)': 698.0,
 'US Recovered': 348.0,
 'US Recovered (+)': 227.0,
 'since': Timestamp('2020-03-20 00:00:00'),
 'updated': Timestamp('2020-03-25 00:00:00')}

In [0]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,2,0,0,4,5,5,1,0,0,2,0,16,0,34,10
Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,2,11,10,5,4,9,4,4,5,6,6,13,15,19,23
Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,2,7,0,5,0,2,1,0,0,4,2,11,11,6,6,14,13,3,49,62,29,34,38
Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,37,0,14,22,13,25,20,31,24
Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0


In [0]:
df_table.head()

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81661.0,3285.0,73280.0,81250.0,3253.0,70535.0,411.0,32.0,2745.0,4.0,Asia
1,Italy,74386.0,7503.0,8326.0,47021.0,4032.0,4440.0,27365.0,3471.0,3886.0,10.1,Europe
2,US,65778.0,942.0,348.0,19100.0,244.0,121.0,46678.0,698.0,227.0,1.4,North America
3,Spain,49515.0,3647.0,3794.0,20410.0,1043.0,1107.0,29105.0,2604.0,2687.0,7.4,Europe
4,Germany,37323.0,206.0,3243.0,19848.0,67.0,113.0,17475.0,139.0,3130.0,0.6,Europe


In [0]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table[df_table['Country/Region'].isin(['China', 'US'])],  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

In [0]:
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:

#hide
from pathlib import Path
if not Path('covid_overview.py').exists():
    ! wget https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/covid_overview.py

--2020-03-26 17:14:45--  https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/covid_overview.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3810 (3.7K) [text/plain]
Saving to: ‘covid_overview.py’

covid_overview.py   100%[===================>]   3.72K  --.-KB/s    in 0s      

2020-03-26 17:14:46 (38.5 MB/s) - ‘covid_overview.py’ saved [3810/3810]



In [0]:
import covid_overview as covid

COL_REGION = 'Province/State'   
# Confirmed, Recovered, Deaths
US_POI = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
    'Colorado', 'Connecticut', 'Delaware', 'Diamond Princess',
    'District of Columbia', 'Florida', 'Georgia', 'Grand Princess',
    'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
    'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
    'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
    'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
    'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming']

filter_us = lambda d: d[d['Country/Region'].eq('US') & d['Province/State'].isin(US_POI)]

kpis_info = [
    {'title': 'New York', 'prefix': 'NY'},
    {'title': 'Washington', 'prefix': 'WA'},
    {'title': 'California', 'prefix': 'CA'}]

data = covid.gen_data(region=COL_REGION, filter_frame=filter_us, kpis_info=kpis_info)

HTTPError: ignored

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)